In [48]:
import os
import sys
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import make_scorer, precision_recall_curve, auc
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
#from sklearn.experimental import enable_halving_search_cv 
#from sklearn.model_selection import HalvingRandomSearchCV
#from sklearn.model_selection import HalvingGridSearchCV 
from collections import Counter
from imblearn.over_sampling import SMOTE
from scipy.stats import ks_2samp
def ks_stat(y, yhat):
    return ks_2samp(yhat[y==1], yhat[y!=1]).statistic

# Menu

<a name="navegacao"></a>

## 1) [Preparação dos dados](#parte1)
- 1.1 [Leitura base principal](#principal)
- 1.2 [Leitura base mes](#mes)
- 1.3 [Leitura base hora](#hora)
- 1.4 [Merge principal e base mensal](#merge1)
- 1.5 [Merge principal e base hora](#merge2)
- 1.6 [Confere marcação](#marcacao)


## 2 [Salvando as bases de treino](#parte2)
- 2.1 [Salvando base com histórico](#comhist)
- 2.2 [Salvando base sem histórico](#semhist)


<a name="principal"></a>

## 1.1) Leitura base principal


## Leitura dos dados

In [2]:
%%time
df = pd.read_csv("BNDES_UNIFICADO.csv",converters={'CNPJ8': str,'INTERMEDIARIA': str},
                 delimiter=";" , encoding='latin-1')

CPU times: total: 422 ms
Wall time: 444 ms


In [3]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [4]:
df.shape

(125516, 27)

In [5]:
df.columns

Index(['CNPJ8', 'ANO', 'EMPRESA_PUBLICA', 'GARANTIA', 'INDIRETA', 'INOVACAO',
       'INSTRUMENTO', 'UF', 'INTERMEDIARIA', 'CUSTO', 'MEDIA_JUROS',
       'PRAZO_AMORTIZACAO', 'TESOURO', 'PRAZO_CARENCIA', 'VALOR_CONTRATO',
       'VALOR_DESENBOLSO', 'Porte_Cliente', 'CAPITAL_SOCIAL', 'IDADE',
       'NATJUR', 'NCONTRATOS', 'NFILIAIS', 'PORTE_RECEITA', 'SITUACAO',
       'IDADE_SOCIOS', 'QTDSOCIOS', 'SOCIO_PJ'],
      dtype='object')

In [6]:
%%time
df.head()

CPU times: total: 0 ns
Wall time: 0 ns


,CNPJ8,ANO,EMPRESA_PUBLICA,GARANTIA,INDIRETA,INOVACAO,INSTRUMENTO,UF,INTERMEDIARIA,CUSTO,...,CAPITAL_SOCIAL,IDADE,NATJUR,NCONTRATOS,NFILIAIS,PORTE_RECEITA,SITUACAO,IDADE_SOCIOS,QTDSOCIOS,SOCIO_PJ
0,00000000,2002,0,OUTROS,0,1,OUTROS,IE,00000000,TJLP,...,9.000002e+10,37.0,2.0,15,5089,5.0,1,1.0,1,0
1,00000000,2003,0,OUTROS,0,0,OUTROS,IE,92816560,TJLP,...,9.000002e+10,38.0,2.0,2,5123,5.0,1,1.0,1,0
2,00000000,2009,1,SEM GARANTIA,0,0,OUTROS,RJ,,TAXAFIXA,...,9.000002e+10,44.0,2.0,5,6912,5.0,1,1.0,1,0
3,00000000,2010,1,SEM GARANTIA,0,0,OUTROS,RJ,,TAXAFIXA,...,9.000002e+10,45.0,2.0,2,7002,5.0,1,1.0,1,0
4,00000000,2012,1,MISTA,0,0,OUTROS,IE,,TJLP,...,9.000002e+10,47.0,2.0,1,7407,5.0,1,1.0,1,0


In [8]:
%%time
df.dtypes

CPU times: total: 0 ns
Wall time: 999 µs


CNPJ8                 object
ANO                    int64
EMPRESA_PUBLICA        int64
GARANTIA              object
INDIRETA               int64
INOVACAO               int64
INSTRUMENTO           object
UF                    object
INTERMEDIARIA         object
CUSTO                 object
MEDIA_JUROS          float64
PRAZO_AMORTIZACAO    float64
TESOURO                int64
PRAZO_CARENCIA       float64
VALOR_CONTRATO       float64
VALOR_DESENBOLSO     float64
Porte_Cliente         object
CAPITAL_SOCIAL       float64
IDADE                float64
NATJUR               float64
NCONTRATOS             int64
NFILIAIS               int64
PORTE_RECEITA        float64
SITUACAO               int64
IDADE_SOCIOS         float64
QTDSOCIOS              int64
SOCIO_PJ               int64
dtype: object

In [9]:
pd.crosstab(df.PORTE_RECEITA,df.SITUACAO)

SITUACAO,0,1
PORTE_RECEITA,,
1.0,72792,2639
3.0,22618,1864
5.0,21244,4359


In [10]:
pd.crosstab(df.Porte_Cliente,df.SITUACAO)

SITUACAO,0,1
Porte_Cliente,,
GRANDE,3692,1219
MICRO,40280,1110
MÉDIA,20556,3430
PEQUENA,52126,3103


In [11]:
%%time
pd.crosstab(df.CUSTO, df.SITUACAO )

CPU times: total: 46.9 ms
Wall time: 57.8 ms


SITUACAO,0,1
CUSTO,,
CDI,14,2
IPCA,48,8
OUTROS,248,119
SELIC,28761,1823
TAXAFIXA,30327,1840
TJLP,20437,2296
TLP,36819,2774


In [12]:
%%time
pd.crosstab(df.NATJUR, df.SITUACAO)

CPU times: total: 31.2 ms
Wall time: 27.2 ms


SITUACAO,0,1
NATJUR,,
1.0,184,49
2.0,116254,8786
3.0,187,21
4.0,29,6


In [13]:
pd.crosstab(df.EMPRESA_PUBLICA, df.SITUACAO)

SITUACAO,0,1
EMPRESA_PUBLICA,,
0,116375,8770
1,279,92


In [14]:
pd.crosstab(df.INDIRETA, df.SITUACAO)

SITUACAO,0,1
INDIRETA,,
0,2711,792
1,113943,8070


In [15]:
pd.crosstab(df.INOVACAO, df.SITUACAO)

SITUACAO,0,1
INOVACAO,,
0,116310,8730
1,344,132


In [16]:
pd.crosstab(df.TESOURO, df.SITUACAO)

SITUACAO,0,1
TESOURO,,
0,110155,7886
1,6499,976


In [17]:
pd.crosstab(df.SOCIO_PJ, df.SITUACAO)

SITUACAO,0,1
SOCIO_PJ,,
0,116654,8862


In [18]:
%%time
df.loc[ (df['CAPITAL_SOCIAL']<1)     ,'CAPITAL_SOCIAL']    = 1
df.loc[ (df['IDADE']<1)              ,'IDADE']             = 1
df.loc[ (df['NCONTRATOS']<1)         ,'NCONTRATOS']        = 1
df.loc[ (df['NFILIAIS']<1)           ,'NFILIAIS']          = 1
df.loc[ (df['IDADE_SOCIOS']<1)       ,'IDADE_SOCIOS']      = 1
df.loc[ (df['QTDSOCIOS']<1)          ,'QTDSOCIOS']         = 1
df.loc[ (df['MEDIA_JUROS']<1)        ,'MEDIA_JUROS']       = 1
df.loc[ (df['PRAZO_AMORTIZACAO']<1)  ,'PRAZO_AMORTIZACAO'] = 1
df.loc[ (df['PRAZO_CARENCIA']<1)     ,'PRAZO_CARENCIA']    = 1
df.loc[ (df['VALOR_CONTRATO']<1)     ,'VALOR_CONTRATO']    = 1
df.loc[ (df['VALOR_DESENBOLSO']<1)   ,'VALOR_DESENBOLSO']  = 1

CPU times: total: 15.6 ms
Wall time: 11 ms


In [19]:
%%time
df['enc_NATJUR']        = df.NATJUR.astype("category").cat.codes
df['enc_GARANTIA']      = df.GARANTIA.astype("category").cat.codes
df['enc_INSTRUMENTO']   = df.INSTRUMENTO.astype("category").cat.codes
df['enc_CUSTO']         = df.CUSTO.astype("category").cat.codes
df['enc_PORTE_CLIENTE'] = df.Porte_Cliente.astype("category").cat.codes
df['enc_PORTE_RECEITA'] = df.PORTE_RECEITA.astype("category").cat.codes
df['enc_SITUACAO']      = df.SITUACAO.astype("category").cat.codes
df['enc_UF']            = df.UF.astype("category").cat.codes

CPU times: total: 46.9 ms
Wall time: 54.7 ms


In [20]:
%%time
df['ln_capsoc']        = np.log(df['CAPITAL_SOCIAL']+1)
df['ln_idade']         = np.log(df['IDADE']+1)
df['ln_contratos']     = np.log(df['NCONTRATOS']+1)
df['ln_filiais']       = np.log(df['NFILIAIS']+1)
df['ln_sociosage']     = np.log(df['IDADE_SOCIOS']+1)
df['ln_qtdsocios']     = np.log(df['QTDSOCIOS']+1)
df['ln_juros']         = np.log(df['MEDIA_JUROS']+1)
df['ln_amortizacao']   = np.log(df['PRAZO_AMORTIZACAO']+1)
df['ln_carencia']      = np.log(df['PRAZO_CARENCIA']+1)
df['ln_vlrcontrato']   = np.log(df['VALOR_CONTRATO']+1)
df['ln_vlrdesembolso'] = np.log(df['VALOR_DESENBOLSO']+1)

CPU times: total: 46.9 ms
Wall time: 26.7 ms


## Modelo sem histórico foi treinado com as variáveis na seguinte ordem:
['faixa_hora', 'vl_medio_mes_atual', 'dif_vl_1', 'tres_prim_dig_codbarras', 'pagador_pf', 'dif_vl_4', 'dia_do_mes', 'qtd_operacoes_mes_corrente', 'vl_medio_dia_corrente', 'sec_dig', 'qtd_operacoes_dia_corrente', 'qtd_trn_60min', 'centavos', 'dia_da_semana']

In [21]:
%%time
files = df.columns
#selected_files = files.str.contains('ln_|enc_|INDIRETA|EMPRESA_PUBLICA|INOVACAO|TESOURO|SOCIO_PJ')
selected_files = files.str.contains('ln_|INDIRETA|EMPRESA_PUBLICA|INOVACAO|TESOURO|SOCIO_PJ')
atributes = files[selected_files]

CPU times: total: 0 ns
Wall time: 997 µs


In [22]:
atributes

Index(['EMPRESA_PUBLICA', 'INDIRETA', 'INOVACAO', 'TESOURO', 'SOCIO_PJ',
       'ln_capsoc', 'ln_idade', 'ln_contratos', 'ln_filiais', 'ln_sociosage',
       'ln_qtdsocios', 'ln_juros', 'ln_amortizacao', 'ln_carencia',
       'ln_vlrcontrato', 'ln_vlrdesembolso'],
      dtype='object')

In [23]:
df[atributes].dtypes

EMPRESA_PUBLICA       int64
INDIRETA              int64
INOVACAO              int64
TESOURO               int64
SOCIO_PJ              int64
ln_capsoc           float64
ln_idade            float64
ln_contratos        float64
ln_filiais          float64
ln_sociosage        float64
ln_qtdsocios        float64
ln_juros            float64
ln_amortizacao      float64
ln_carencia         float64
ln_vlrcontrato      float64
ln_vlrdesembolso    float64
dtype: object

In [24]:
df[atributes].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
EMPRESA_PUBLICA,125516.0,0.002956,0.054287,0.000000,0.000000,0.000000,0.000000,1.000000
INDIRETA,125516.0,0.972091,0.164712,0.000000,1.000000,1.000000,1.000000,1.000000
INOVACAO,125516.0,0.003792,0.061465,0.000000,0.000000,0.000000,0.000000,1.000000
TESOURO,125516.0,0.059554,0.236660,0.000000,0.000000,0.000000,0.000000,1.000000
SOCIO_PJ,125516.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ln_capsoc,125516.0,10.674521,3.548741,0.693147,9.615872,11.002117,11.982935,26.048381
ln_idade,125516.0,1.250731,0.858364,0.693147,0.693147,0.693147,1.791759,4.718499
ln_contratos,125516.0,0.813766,0.313496,0.693147,0.693147,0.693147,0.693147,5.613128
ln_filiais,125516.0,0.876072,0.495155,0.693147,0.693147,0.693147,0.693147,8.921591
ln_sociosage,125516.0,2.572323,1.539442,0.693147,0.693147,3.610918,3.850148,4.510860


## SMOTE: Synthetic Minority Oversampling Technique

In [25]:
y0 = df['SITUACAO'].values.reshape(-1, 1)
#y0 = df['SITUACAO'].values
X0 = df[atributes]

In [26]:
#import joblib
#scaler = joblib.load("scaler.saved") 

from numpy import asarray
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# transform data
X0 = scaler.fit_transform(df[atributes])

In [27]:
# base sintética: treino e teste
x_train  , x_test0 , y_train, y_test0 = train_test_split(X0, y0, test_size = 0.4, random_state=123)

# base sintética: teste e out of sample
x_test , x_out , y_test, y_out = train_test_split(x_test0, y_test0, test_size = 0.4, random_state=123)

In [28]:
# summarize the new class distribution
#counter0 = Counter(y_train)
#counter1 = Counter(y_test)
#counter2 = Counter(y_out)
#print(counter0, counter1, counter2)

In [34]:
%%time
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

gr_range = ['depthwise','lossguide']
bs_range = ['gbtree','dart']

def gbm_xgb(learning_rate, max_depth, gamma, min_child_weight, subsample, eta, reg_alpha, reg_lambda,
            n_estimators, max_delta_step, max_leaves, grow_policy, scale_pos_weight, booster , max_bin):
    model = xgb.XGBClassifier(
                  learning_rate       = learning_rate,
                  max_depth           = int(max_depth),
                  gamma               = gamma,
                  min_child_weight    = int(min_child_weight),
                  subsample           = subsample,
                  eta                 = eta,
                  reg_alpha           = reg_alpha,
                  reg_lambda          = reg_lambda,
                  n_estimators        = int(n_estimators),
                  max_delta_step      = max_delta_step,
                  max_leaves          = int(max_leaves),
                  max_bin             = int(max_bin),
                  booster             = bs_range[int(booster)],
                  grow_policy         = gr_range[int(grow_policy)],
                  scale_pos_weight    = scale_pos_weight,
                  missing             = 0,
                  random_state        = 666,
                  nthread =10 )
    
    model.fit(x_train, y_train.ravel(), verbose=False)
    pred_labels = model.predict(x_test)
    return roc_auc_score(y_test, pred_labels)

params_xgb = {
    'learning_rate'          : (0.010 ,0.500),
    'max_depth'              : (2.000 ,11.00),
    'gamma'                  : (1.000 ,100.0),
    'min_child_weight'       : (1.000 ,100.0),
    'subsample'              : (0.222 ,0.999),
    'eta'                    : (0.005 ,0.500),
    'reg_alpha'              : (0.050 ,10.00),
    'reg_lambda'             : (0.050 ,10.00),
    'n_estimators'           : (50.00 ,500.0),
    'max_delta_step'         : (0.005 ,5.000),
    'max_leaves'             : (2.000 ,50.00),
    'max_bin'                : (2.000 ,100.0),
    'booster'                : (0.001 ,1.000),
    'grow_policy'            : (0.001 ,1.000),
    'scale_pos_weight'       : (0.222 ,100.0),
}
   
xgb0 = BayesianOptimization(f=gbm_xgb, pbounds=params_xgb, random_state=123, allow_duplicate_points=True) 
xgb0.set_gp_params(alpha=1e-4)
xgb0.maximize(init_points=30, n_iter=270)

|   iter    |  target   |  booster  |    eta    |   gamma   | grow_p... | learni... |  max_bin  | max_de... | max_depth | max_le... | min_ch... | n_esti... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.9       | 0.6968    | 0.1466    | 23.46     | 0.5518    | 0.3625    | 43.46     | 4.904     | 8.163     | 25.08     | 39.82     | 204.4     | 7.304     | 4.414     | 6.177     | 0.5313    |
| 2         | 0.9076    | 0.7383    | 0.09533   | 18.37     | 0.532     | 0.2706    | 64.17     | 4.248     | 8.52      | 31.33     | 72.52     | 195.3     | 3.65      | 2.321     | 29.53     | 0.7123    |
| 3         | 0.9063    | 0.09301   | 0.2197    | 43.66     | 0.4942    | 0.2187    | 32.6      | 2.135     | 10.04     | 47.32     | 50.68     | 330.8     | 1.2       | 3.207 

In [35]:
params_xgb = xgb0.max['params']
params_xgb

{'booster': 0.7112876949019146,
 'eta': 0.4856678395525138,
 'gamma': 87.29661038108841,
 'grow_policy': 0.7104514896693779,
 'learning_rate': 0.47966977408061673,
 'max_bin': 44.121707112885026,
 'max_delta_step': 4.365030177037427,
 'max_depth': 5.203619011762532,
 'max_leaves': 46.62865533938464,
 'min_child_weight': 15.728987967968257,
 'n_estimators': 473.0130567177231,
 'reg_alpha': 8.335526162967966,
 'reg_lambda': 8.468245640054818,
 'scale_pos_weight': 12.586790084191318,
 'subsample': 0.6854703200115296}

In [41]:
learning_rate       = xgb0.max['params']['learning_rate']
max_depth           = int(xgb0.max['params']['max_depth'])
gamma               = xgb0.max['params']['gamma']
min_child_weight    = int(xgb0.max['params']['min_child_weight'])
subsample           = xgb0.max['params']['subsample']
eta                 = xgb0.max['params']['eta']
reg_alpha           = xgb0.max['params']['reg_alpha']
reg_lambda          = xgb0.max['params']['reg_lambda']
n_estimators        = int(xgb0.max['params']['n_estimators'])
max_delta_step      = xgb0.max['params']['max_delta_step']
max_leaves          = int(xgb0.max['params']['max_leaves'])
max_bin             = int(xgb0.max['params']['max_bin'])
booster             = bs_range[int(xgb0.max['params']['booster'])]
grow_policy         = gr_range[int(xgb0.max['params']['grow_policy'])]
scale_pos_weight    = xgb0.max['params']['scale_pos_weight']

print('\n learning_rate:'     , learning_rate,
      '\n max_depth:'         , max_depth,
      '\n gamma:'             , gamma,
      '\n min_child_weight:'  , min_child_weight,
      '\n subsample:'         , subsample,
      '\n eta:'               , eta,
      '\n reg_alpha:'         , reg_alpha,
      '\n reg_lambda:'        , reg_lambda,
      '\n n_estimators:'      , n_estimators,
      '\n max_delta_step:'    , max_delta_step,
      '\n max_leaves:'        , max_leaves,
      '\n max_bin:'           , max_bin,
      '\n grow_policy:'       , grow_policy,
      '\n booster:'           , booster,
      '\n scale_pos_weight:'  , scale_pos_weight)


 learning_rate: 0.47966977408061673 
 max_depth: 5 
 gamma: 87.29661038108841 
 min_child_weight: 15 
 subsample: 0.6854703200115296 
 eta: 0.4856678395525138 
 reg_alpha: 8.335526162967966 
 reg_lambda: 8.468245640054818 
 n_estimators: 473 
 max_delta_step: 4.365030177037427 
 max_leaves: 46 
 max_bin: 44 
 grow_policy: depthwise 
 booster: gbtree 
 scale_pos_weight: 12.586790084191318


In [42]:
%%time
cbbb= xgb.XGBClassifier(learning_rate     = learning_rate,
                        max_depth         = max_depth,
                        gamma             = gamma,
                        min_child_weight  = min_child_weight,
                        subsample         = subsample,
                        eta               = eta,
                        reg_alpha         = reg_alpha,
                        reg_lambda        = reg_lambda,
                        n_estimators      = n_estimators,
                        max_delta_step    = max_delta_step,
                        max_leaves        = max_leaves,
                        max_bin           = max_bin,
                        grow_policy       = grow_policy,
                        booster           = booster,
                        scale_pos_weight  = scale_pos_weight)
cbbb.fit(x_train, y_train)

c:\Users\patrick.franco\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\patrick.franco\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


CPU times: total: 1min 40s
Wall time: 14.3 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.4856678395525138,
              gamma=87.29661038108841, gpu_id=-1, grow_policy='depthwise',
              importance_type='gain', interaction_constraints='',
              learning_rate=0.47966977408061673, max_bin=44,
              max_delta_step=4.365030177037427, max_depth=5, max_leaves=46,
              min_child_weight=15, missing=nan, monotone_constraints='()',
              n_estimators=473, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=8.335526162967966, reg_lambda=8.468245640054818,
              scale_pos_weight=12.586790084191318, subsample=0.6854703200115296,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [65]:
%%time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
print("Accuracy:" , accuracy_score(  y_train, cbbb.predict(x_train))) 
print("F1 score:" , f1_score(        y_train, cbbb.predict(x_train))) 
print("Recall:"   , recall_score(    y_train, cbbb.predict(x_train))) 
print("Precision:", precision_score( y_train, cbbb.predict(x_train))) 
print("KS score:" , ks_stat(         y_train.ravel(), cbbb.predict_proba(x_train)[:,1] ))
print("AUC score:", roc_auc_score(   y_train, cbbb.predict_proba(x_train)[:,1]))

Accuracy: 0.8586888685283299
F1 score: 0.4926098979689139
Recall: 0.9643457158857569
Precision: 0.3307933662034962
KS score: 0.817645390717586
AUC score: 0.9408950451700975
CPU times: total: 2.73 s
Wall time: 592 ms


In [66]:
%%time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print("Accuracy:" ,accuracy_score(  y_test, cbbb.predict(x_test))) 
print("F1 score:" ,f1_score(        y_test, cbbb.predict(x_test))) 
print("Recall:"   ,recall_score(    y_test, cbbb.predict(x_test))) 
print("Precision:",precision_score( y_test, cbbb.predict(x_test))) 
print("KS score:" , ks_stat(        y_test.ravel(), cbbb.predict_proba(x_test)[:,1] ))
print("AUC score:", roc_auc_score(  y_test, cbbb.predict_proba(x_test)[:,1]))

Accuracy: 0.856327180985261
F1 score: 0.4869606448553817
Recall: 0.9706994328922496
Precision: 0.325
KS score: 0.8185643286363227
AUC score: 0.9387236157614163
CPU times: total: 1.14 s
Wall time: 213 ms


In [67]:
%%time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print("Accuracy:" ,accuracy_score(  y_out, cbbb.predict(x_out))) 
print("F1 score:" ,f1_score(        y_out, cbbb.predict(x_out))) 
print("Recall:"   ,recall_score(    y_out, cbbb.predict(x_out))) 
print("Precision:",precision_score( y_out, cbbb.predict(x_out))) 
print("KS score:" , ks_stat(        y_out.ravel(), cbbb.predict_proba(x_out)[:,1] ))
print("AUC score:", roc_auc_score(  y_out, cbbb.predict_proba(x_out)[:,1]))

Accuracy: 0.8586864512274063
F1 score: 0.4866136034732272
Recall: 0.9683225341972642
Precision: 0.3249577192558589
KS score: 0.8198992866277341
AUC score: 0.9390985107197629
CPU times: total: 750 ms
Wall time: 149 ms


## Leitura dos dados originais

In [68]:
%%time
df['proba'] = cbbb.predict_proba(X0)[:,1]

CPU times: total: 531 ms
Wall time: 129 ms


In [69]:
df[['SITUACAO','proba']].groupby(['SITUACAO']).count()

,proba
SITUACAO,
0,116654
1,8862


In [70]:
%%time
df[['SITUACAO','proba']].groupby(['SITUACAO']).describe()

CPU times: total: 31.2 ms
Wall time: 25.9 ms


proba                                                    \
             count      mean       std       min       25%       50%   
SITUACAO                                                               
0         116654.0  0.151084  0.292995  0.001301  0.007122  0.011834   
1           8862.0  0.844095  0.132819  0.004927  0.842715  0.878845   

                              
               75%       max  
SITUACAO                      
0         0.063389  0.953062  
1         0.903306  0.953062

In [83]:
%%time
df.loc[ df['proba']>=0.84 ,'PRED'] = 1
df.loc[ df['proba'] <0.84 ,'PRED'] = 0

CPU times: total: 0 ns
Wall time: 2 ms


<a name="metricas"></a>


# 3) Métricas
  
- ir para [Menu Principal](#navegacao)

<a name="amostra"></a>


## 3.1) Métricas na Amostra
  
- ir para [Menu Principal](#navegacao)

In [84]:
from sklearn.metrics import confusion_matrix
confusao_pop = confusion_matrix(df['SITUACAO'], df['PRED'])
print("Confusion matrix for test:\n%s" % confusao_pop )

Confusion matrix for test:
[[106178  10476]
 [  2152   6710]]


In [85]:
%%time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

print("Accuracy:" , accuracy_score(df['SITUACAO'], df['PRED'])) 
print("F1 score:" , f1_score(df['SITUACAO'], df['PRED'])) 
print("Recall:"   , recall_score(df['SITUACAO'], df['PRED'])) 
print("Precision:", precision_score(df['SITUACAO'], df['PRED'])) 

Accuracy: 0.899391312661334
F1 score: 0.5152027027027026
Recall: 0.7571654254118709
Precision: 0.3904340742464797
CPU times: total: 156 ms
Wall time: 144 ms


<a name="falsopos"></a>

## 3.2) Taxa de Falso Positivo
  
- ir para [Menu Principal](#navegacao)

In [86]:
%%time
tn, fp, fn, tp = confusao_pop.ravel()
print('FPR:', fp/(fp + tn))
print('TPR:', tp/(tp + fn))

FPR: 0.08980403586675124
TPR: 0.7571654254118709
CPU times: total: 0 ns
Wall time: 998 µs


In [87]:
metrics_list_safra = []
safras = df['ANO'].unique()
safras.sort()
i=0
for ano in safras:
    metrics = dict()
    metrics['Model Index']=i
    metrics['ANO']=ano
    valid_hue = df[df['ANO']==ano]
    print(ano)
    confusao_pop = confusion_matrix(valid_hue['SITUACAO'], valid_hue['PRED'])
    
    metrics["Accuracy"]  = accuracy_score(valid_hue['SITUACAO'] , valid_hue['PRED'])
    metrics['AUC']       = roc_auc_score(valid_hue['SITUACAO']  , valid_hue['proba'])
    metrics['KS Score']  = ks_stat(valid_hue['SITUACAO']        , valid_hue['proba'])
    metrics["F1 score"]  = f1_score(valid_hue['SITUACAO']       , valid_hue['PRED'])
    metrics["Recall"]    = recall_score(valid_hue['SITUACAO']   , valid_hue['PRED']) 
    metrics["Precision"] = precision_score(valid_hue['SITUACAO'], valid_hue['PRED'])
    metrics["False positive rate"] = confusao_pop[0][1]/(confusao_pop[0][0]+confusao_pop[0][1])
    metrics["True positive rate"]  = confusao_pop[1][1]/(confusao_pop[1][0]+confusao_pop[1][1])
    metrics_list_safra.append(metrics)
    del valid_hue, confusao_pop

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [88]:
df_metrics = pd.DataFrame(metrics_list_safra)

In [89]:
df_metrics.to_csv('df_metrics_xbt.csv')

In [90]:
df_metrics

,Model Index,ANO,Accuracy,AUC,KS Score,F1 score,Recall,Precision,False positive rate,True positive rate
0,0,2002,0.881579,0.917424,0.739394,0.929134,0.893939,0.967213,0.200000,0.893939
1,0,2003,0.833333,0.895508,0.625000,0.882353,0.937500,0.833333,0.375000,0.937500
2,0,2004,0.815385,0.920000,0.709524,0.833333,0.857143,0.810811,0.233333,0.857143
3,0,2005,0.620370,0.721301,0.453514,0.506024,0.724138,0.388889,0.417722,0.724138
4,0,2006,0.741007,0.828965,0.699115,0.538462,0.807692,0.403846,0.274336,0.807692
5,0,2007,0.668639,0.822368,0.573726,0.500000,0.777778,0.368421,0.360902,0.777778
6,0,2008,0.725275,0.814725,0.621249,0.545455,0.810811,0.410959,0.296552,0.810811
7,0,2009,0.739464,0.873712,0.645749,0.558442,0.826923,0.421569,0.282297,0.826923
8,0,2010,0.692661,0.833547,0.524784,0.608187,0.881356,0.464286,0.377358,0.881356
9,0,2011,0.725490,0.827086,0.588235,0.539474,0.803922,0.405941,0.294118,0.803922


In [91]:

dataframe=pd.DataFrame(cbbb.feature_importances_, columns=['feature_importances'])
dataframe['features'] = list(atributes)
dataframe.sort_values('feature_importances',inplace=True,ascending=False)
dataframe

,feature_importances,features
8,0.849262,ln_filiais
6,0.037258,ln_idade
5,0.020132,ln_capsoc
11,0.016954,ln_juros
7,0.016457,ln_contratos
9,0.014505,ln_sociosage
14,0.012706,ln_vlrcontrato
12,0.011513,ln_amortizacao
13,0.009917,ln_carencia
15,0.005734,ln_vlrdesembolso


In [92]:
dataframe.to_csv('feature_importances_xbt.csv')

In [94]:
df.proba.quantile([0.01,0.025,0.05,0.10,0.25,0.35,0.50,0.99])

0.010    0.002697
0.025    0.002965
0.050    0.004050
0.100    0.004849
0.250    0.007389
0.350    0.008677
0.500    0.013067
0.990    0.924329
Name: proba, dtype: float64

In [ ]:
def risco(row):
    if row['proba']>=0.389181:
        return 1
    elif row['proba']>=0.312434:
        return 2
    elif row['proba']>=0.232285:
        return 3
    elif row['proba']>=0.202390:
        return 4
    elif row['proba']>=0.009928:
        return 5
    elif row['proba']>=0.001094:
        return 6
    elif row['proba']>=0.000795:
        return 7
    elif row['proba']>=0.000570:
        return 8
    else:
        return 9

df['nivel_risco'] = df.apply(risco, axis=1) 